In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
df = pd.read_csv('C:/Users/DELL/OneDrive/Desktop/data.csv')
df.head()

,subreddit,post_id,sentence_range,text,label,syntax_fk_grade,Stress Level
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,7.910952,-0.204167


In [3]:
new_df = pd.DataFrame(columns=['text', 'label'])
new_df['text'] = df["text"]
new_df['label'] = df["label"]
new_df.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


# Multinomial Naive Bayes

In [4]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new_df['text'], new_df['label'], test_size=0.1,shuffle=True, random_state=0)

# create a bag-of-words model
vectorizer = CountVectorizer(stop_words='english')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)

# predict stress levels for the test set
y_pred = clf.predict(X_test_bow)

# calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.75


# Logistic Regression

In [5]:
vectorizer = CountVectorizer(stop_words='english')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# train the logistic regression model
lr = LogisticRegression()
lr.fit(X_train_bow, y_train)

# evaluate the model
y_pred = lr.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7394366197183099


In [6]:
# initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

In [14]:
def detect_stress_level(text):
    # analyze the sentiment of the text
    sentiment = sia.polarity_scores(text)
    # get the compound score
    compound_score = sentiment['neg']
    
    # check if any stress words are present in the text
    msg_bow = vectorizer.transform([text])
    # predict the stress level using the ML model
    stress_present = clf.predict(msg_bow)[0]
    
    # determine the stress level based on compound score and stress word presence
    if compound_score < -0.5 and stress_present:
        stress_level = 10
    elif compound_score < -0.3 and stress_present:
        stress_level = 8
    elif compound_score < -0.1 and stress_present:
        stress_level = 6
    elif compound_score < 0.1 and stress_present:
        stress_level = 4
    elif compound_score < 0.3 and stress_present:
        stress_level = 2
    else:
        stress_level = 0
    # return the stress level
    return stress_level

In [ ]:
while True:
    # get user input
    user_input = input("You: ")
    # generate a response based on the user input
    stress_level = detect_stress_level(user_input)
    # print the response
    if stress_level == 10:
        response = "It sounds like you're experiencing a lot of stress. Have you considered talking to a therapist or seeking professional help?"
    elif stress_level == 8:
        response = "It seems like you're feeling pretty stressed. Maybe try taking some deep breaths or going for a walk to clear your mind."
    elif stress_level == 6:
        response = "You seem to be feeling stressed. Is there anything specific that's causing you to feel this way?"
    elif stress_level == 4:
        response = "You seem to be feeling somewhat stressed. Sometimes it can help to take a break and do something you enjoy."
    elif stress_level == 2:
        response = "It seems like you are little bit stressed. Take some rest and this will help you to regain your energy."
    elif stress_level == 0:
        response = "It sounds like you're feeling pretty good at the moment. Is there anything you'd like to talk about?"
    
    print("Bot:", stress_level)
    print(response)